# Document Analysis using flan-t5 model and langchain framework with trained dataset 

In [2]:
!pip install --upgrade pip
!pip install --upgrade sagemaker --quiet
!pip install ipywidgets==7.0.0 --quiet
!pip install langchain==0.0.148 --quiet
!pip install faiss-cpu --quiet
!pip install pytesseract --quiet
!pip install unstructured --quiet
!pip install transformers --quiet
!pip install datasets --quiet
!pip install pdfreader
!pip install tiktoken
!pip install openai
!pip install PyPDF2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.15.

# Finetue Flan-T5 using sagemaker
Train and deploy the flan-t5 model using sagemaker jumpstart UI.
Billsum datset we have used here is copied to S3 bucket. And using the dataset to train the model.

In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


Note down the endpoint name from jumpstart UI and Provide the endpoint name below

In [5]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain import PromptTemplate

parameters = {
    "max_length": 200,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": True,
    "temperature": 1,
    
}

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_texts"][0]


content_handler = ContentHandler()

sm_llm = SagemakerEndpoint(
    # endpoint_name=_MODEL_CONFIG_["huggingface-text2text-flan-t5-xl"]["endpoint_name"],
    endpoint_name="jumpstart-ftc-hf-text2text-flan-t5-xl",
    region_name=aws_region,
    model_kwargs=parameters,
    content_handler=content_handler,
)

Copy the document in the local folder. 
Give the Document name as input in the below variable 'License_Agreement'

In [6]:
from PyPDF2 import PdfReader
# s3_client = boto3.client('s3')
# bucket_name = 'sagemaker-soln-documents-js-lplljf-867164791015-us-east-1'
# object_key = 'Agreements/Agreement1.pdf'
License_Agreement = 'sample5.pdf'  # Choose a local path for the downloaded file
# s3_client.download_file(bucket_name, object_key, License_Agreement)
reader = PdfReader(License_Agreement)

file_content =""
number_of_pages = len(reader.pages)
for p in range(number_of_pages):
    page = reader.pages[p]
    file_content += page.extract_text()
print(file_content[:200])

SECTION 1. SHORT TITLE. This Act may be cited as ``Small Business Tax Modernization Act of 2004''. 
SEC. 2. UNIFIED PASS -THRU ENTITY REGIME. (a) Termination of S Corporation Status. -- (1) No new s 



In [7]:
sm_llm.get_num_tokens(file_content)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (2485 > 1024). Running this sequence through the model will result in indexing errors


2485

In [8]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

The token size of flan-t5 is limited to 1024.
Below is the fuction to adjust the chunk size of the document during splitting which will split the documnet with maximum allowable token size 

In [10]:
chunk_size = 6000
max_chunk_size = 1024  # Maximum allowed chunk size

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=chunk_size, chunk_overlap=0)
docs = text_splitter.create_documents([file_content])
output_docs = []

for i in range(len(docs)):
    doc = docs[i]
    doc_size = sm_llm.get_num_tokens(doc.page_content)
    
    while doc_size > max_chunk_size:
        chunk_size -= 100  # Adjust the chunk size (you can change the adjustment value)
        text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=chunk_size, chunk_overlap=0)
        docs = text_splitter.create_documents([file_content])
        doc = docs[i]
        doc_size = sm_llm.get_num_tokens(doc.page_content)
    
    output_docs.append(doc)
    
for i, doc in enumerate(output_docs):
    print(f"DOC {i}: token size {sm_llm.get_num_tokens(doc.page_content)}")


Token indices sequence length is longer than the specified maximum sequence length for this model (1522 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1498 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1472 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1444 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1411 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

DOC 0: token size 1012
DOC 1: token size 985


In [11]:
map_prompt = """
Please provide a clear and concise summary of the text, ensuring that any terms, conditions, dates(e.g., Expiration Date, Effective Date, Agreement Date), and relevant clauses present in the text are included in the summary:
TEXT:`{text}`
Brief Summary:
"""

map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])


summary_chain = load_summarize_chain(llm=sm_llm, chain_type='map_reduce',
                                        map_prompt=map_prompt_template
                                     #verbose=True # Set verbose=True if you want to see the prompts being used
                                    )

Using langchain framework we are getting the summary of all the splitted document and concatenating to get the desired summary using sagemaker jumpstart.

In [12]:
concatenated_summaries = ""
# count = 0
# Iterate through the documents
for doc, metadata in docs:
    # count += 1
    new_doc_list = list(doc)

    
    # Access the document content from the tuple
    document_content_string = new_doc_list[1]
    
    document_content = text_splitter.create_documents([document_content_string])

#     # Run summarization on the document content
    summary = summary_chain.run(document_content)
    
    # print(type(summary))
    
#     # Append the summary to the concatenated_summaries string
    concatenated_summaries += summary + "\n" 
    # if count == 11:
    #     break


# print(concatenated_summaries)

In [17]:
endpoint_name="jumpstart-ftc-hf-text2text-flan-t5-xl"

In [18]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text

In [19]:
text=concatenated_summaries

Provide various prompts to get the desired summary

In [22]:
prompts = [
    """
    Please provide a clear and concise summary of the text, ensuring that any terms, conditions, dates(e.g.,'8th day of May 2014', 'May 8, 2014'), and relevant clauses present in the text are included in:`{text}` """,

"""Summarize the essential terms, conditions, and obligations outlined in the contract or any session mentioned: TEXT:`{text}`
""",
"Briefly summarize this sentence in 500 words: `{text}`",
    
    
"""Please provide a clear and concise summary of the text, ensuring that any terms, conditions, dates(e.g., Expiration Date, Effective Date, Agreement Date), and relevant clauses present in the text are included in the summary:
TEXT:`{text}`
Brief Summary:
"""


    
    
    # "Briefly summarize this sentence: {text}",
    #  "Write a short summary for this text: {text}",
    # "Generate a short summary this sentence:\n{text}",
    # "{text}\n\nWrite a brief summary in a sentence or less",
    # "{text}\nSummarize the aforementioned text in a single phrase.",
    # "{text}\nCan you generate a short summary of the above paragraph?",
    # "Write a sentence based on this summary: {text}",
    # "Write a sentence based on '{text}'",
    # "Summarize this article:\n\n{text}",
]

num_return_sequences = 1
parameters = {
    "max_length": 3000,
    "max_time": 600,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}

print(f"{bold}Number of return sequences are set as {num_return_sequences}{unbold}{newline}")
for each_prompt in prompts:
    payload = {"text_inputs": each_prompt.replace("{text}", text), **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} For prompt: '{each_prompt}'{unbold}{newline}")
    print(f"{bold} The {num_return_sequences} summarized results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

Number of return sequences are set as 1

 For prompt: '
    Please provide a clear and concise summary of the text, ensuring that any terms, conditions, dates(e.g.,'8th day of May 2014', 'May 8, 2014'), and relevant clauses present in the text are included in:`{text}` '

 The 1 summarized results are:

Result 0: Small Business Tax Modernization Act of 2004 - Amends the Internal Revenue Code to: (1) terminate S corporation tax status for taxable years beginning after 2004; (2) permit certain S corporations to elect to be taxed as partnerships; (3) revise S corporation tax treatment for transactions involving a person who does not own a principal residence at the time of the purchase or sale of the principal residence; (4) allow a greater deduction for net operating losses for S corporations with adjusted gross receipts under $100,000; (5) treat certain reorganized S corporations as reorganized corporations for purposes of S corporation conversion to 

 For prompt: 'Summarize the essenti

### Clean up the endpoint

Delete the SageMaker endpoint using jumpstart UI


In [28]:
!pip list


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Package                              Version
------------------------------------ --------------------
aiohttp                              3.8.6
aiosignal                            1.3.1
alabaster                            0.7.12
anaconda-client                      1.11.0
anaconda-project                     0.11.1
anyio                                3.5.0
appdirs                              1.4.4
argon2-cffi                          21.3.0
argon2-cffi-bindings                 21.2.0
arrow                                1.2.2
astroid                              3.0.0a9
astropy                              5.3.3
asttokens                            2.4.0
async-timeout                        4.0.3
atomicwrites                         1.4.0
attrs                                23.1.0
Automat                              20.2.0
autopep8                             1.6.0
autovizwidget                        0.20.5
awscli                               1.29.42
Babel                    